In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
from warnings import catch_warnings

In [ ]:
import sys

sys.path.append("..")

In [ ]:
# Import the various experiments we need for the notebook
from sva.experiments import Simple2d, CampaignParameters

# Import the seeding function for reproducibility
from sva.utils import seed_everything, set_mpl_defaults

In [ ]:
set_mpl_defaults()

# A simple 2d example

In [ ]:
seed_everything(123)

In [ ]:
experiments = []
acquisition_functions = ["EI", "UCB", "UCB", "UCB"]
acquisition_function_kwargs = [
    {},
    {"beta": 10.0},
    {"beta": 40.0},
    {"beta": 100.0},
]

for acqf, acqf_kwargs in zip(
    acquisition_functions, acquisition_function_kwargs
):
    experiment = Simple2d()
    experiment.initialize_data(n=3, protocol="random")
    acquisition_function = {"method": acqf, "kwargs": acqf_kwargs}
    with catch_warnings(record=True) as thrown_warnings:
        parameters = CampaignParameters(
            acquisition_function=acquisition_function
        )
    experiment.run(100, parameters=parameters)
    experiments.append(experiment)

In [ ]:
# Get the ground truth results
x = experiment.get_dense_coordinates(ppd=100)
y, _ = experiment(x)

In [ ]:
extent = experiment.get_experimental_domain_mpl_extent()

From left to right, we have EI, UCB(10), UCB(40) and UCB(100).

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(8, 3), sharex=True, sharey=True)

for ii, experiment in enumerate(experiments):
    ax = axs[ii]
    X, Y = experiment.data.X, experiment.data.Y
    ax.imshow(
        y.reshape(100, 100).T,
        extent=extent,
        interpolation="nearest",
        origin="lower",
    )
    ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5)

plt.show()